In [1]:
from math import comb

import numpy as np
from numpy.testing import assert_almost_equal, assert_equal
from scipy.stats import t as stats_t, moment
from scipy.stats import ttest_ind

from tqdm import tqdm

In [2]:
from src.tvla.tvla import Tvla, central_moment, Group, central_sum

TRACE_LEN = 1

def make_t_test(n: int):
    """
    Returns a t-test that takes the sample mean and variance for a list of sample points from A, and a list of sample
    points for B.
    """
    def welch_t_test(ma: np.array, va: np.array, mb: np.array, vb: np.array):
        m = ma - mb

        sa = va / n
        sb = vb / n

        sab = sa + sb
        t = m / np.sqrt(sab)

        dof = sab ** 2 / ((sa ** 2 + sb ** 2) / (n - 1))

        p = 2 * stats_t(df=dof).cdf(-np.abs(t))

        return t, p

    return welch_t_test


def gen_example(mean, trace_num, trace_len=TRACE_LEN):
    return np.random.normal(mean, 2.2, size=(trace_num, trace_len)).astype(int)


def get_mv(x: np.array):
    return np.array((x.mean(axis=0, dtype=np.float64), x.var(axis=0, dtype=np.float64)))


num_traces = 1000000

ex_a = gen_example(2, num_traces)
ex_b = gen_example(2, num_traces)

res_sp = ttest_ind(ex_a, ex_b, axis=0, equal_var=False)[1]

test = make_t_test(num_traces)

res_custom = test(*get_mv(ex_a), *get_mv(ex_b))[1]

assert_almost_equal(res_custom, res_sp, decimal=3)

In [3]:
for M in range(1, 10):
    assert_almost_equal(central_moment(ex_a, M), moment(ex_a, M, axis=0))

In [4]:
assert_almost_equal(Group(ex_a).t_test(Group(ex_b), 1)[0], ttest_ind(ex_a, ex_b, axis=0)[0], 2)

$(\frac{n - 1}{n} \Delta) ^ d = 0$ for $n = 1$.

In [5]:
def cm1(m1, y, n):
    delta = y - m1
    return m1 + delta / n

cm1(np.zeros(1), ex_a[0], 1)[0], ex_a[0][0]

(1.0, 1)

In [6]:
class GroupAccu:
    """
    Based on the incremental calculation from the paper.
    """
    def __init__(self, trace_len, max_order=1):
        self.trace_len = trace_len
        self.max_order = max_order + 1
        self.max_computed_order = 2 * max_order + 1

        self.n = 0

        shape = (self.max_computed_order, trace_len)
        self.cs, self.cm = np.zeros(shape), np.zeros(shape)
        self.mean = np.zeros(trace_len)

    def __add_order(self, x, d):
        delta = x - self.mean

        if d == 1:
            self.mean += delta / self.n
        if d >= 2:
            acc = self.cs[d]

            for k in range(1, d - 1):
                acc += comb(d, k) * self.cs[d-k] * (- delta / self.n) ** k

            if self.n > 1:
                acc += ((((self.n-1)/self.n)*delta) ** d) * (1 - (-1/(self.n-1)) ** (d - 1))

            self.cs[d] = acc


    def sm(self, d):
        """
        Estimated Standardized Moment of a given order.
        """
        if d > 2:
            return self.cm[d] / (self.cm[d] ** (d / 2))
        if d == 2:
            return self.cm[d]
        if d == 1:
            return self.cm[d]

    def s2(self, d):
        """
        Estimated Variance of a given order.
        """
        if d == 1:
            return self.cm[2]
        if d == 2:
            return self.cm[4] - (self.cm[2] ** 2)
        if d > 2:
            return (self.cm[d * 2] - (self.cm[d] ** 2)) / (self.cm[2] ** d)

    def __add(self, x):
        self.n += 1

        for d in range(1, self.max_computed_order):
            self.__add_order(x, d)
            self.cm[2:] = self.cs[2:] / self.n

    def add(self, traces):
        skip_first = len(traces) - 1

        if self.n == 0:
            for d in range(2, self.max_computed_order):
                self.mean = traces[:skip_first].mean(axis=0)
                self.cs[d] = central_sum(traces[:skip_first], d)
                self.cm[d] = self.cs[d] / skip_first

            self.n = skip_first
            traces = traces[skip_first:]

        # Function can be reduced to this
        for x in tqdm(traces):
            self.__add(x)

GRP = GroupAccu(1, 3)
NUM = 10
GRP.add(ex_a[:NUM])

assert_almost_equal(GRP.mean, ex_a[:NUM].mean(axis=0))

# TODO it starts to deviate immediately.
GRP.cm[3], moment(ex_a[:NUM], 3)
# And then it converges back to the correct moment.

100%|██████████| 1/1 [00:00<00:00, 718.94it/s]


(array([1.95582864]), array([1.968]))

In [7]:
NUM = 1000
tvla = Tvla(1)

def gen_example_var(variance=2.2, trace_num=1000, trace_len=TRACE_LEN):
    return np.random.normal(2, variance, size=(trace_num, trace_len)).astype(int)

tvla.add(ex_a[:NUM], gen_example_var(3, NUM))

tvla.min_p_order()

  0%|          | 0/1000 [00:00<?, ?it/s]/home/berg/Documents/Afstudeer/LA/src/tvla/tvla.py:23: RuntimeWarning: Mean of empty slice.
  self.mean = traces.mean(axis=0)
/usr/local/lib/python3.8/dist-packages/numpy-1.19.4-py3.8-linux-x86_64.egg/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/berg/Documents/Afstudeer/LA/src/tvla/tvla.py:37: RuntimeWarning: invalid value encountered in double_scalars
  cm = np.sum(mean_free ** order, axis=0) / self.num_traces
  0%|          | 0/1000 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
tvla.p_gradient(2)[-1]